## CS-E4820 Machine Learning: Advanced Probabilistic Methods (spring 2022)

Pekka Marttinen, Prayag Tiwari, Vishnu Raj, Tianyu Cui, Yogesh Kumar, Antti Pöllänen, Louis Filstroff, Alex Aushev, Zheyang Shen, Nikitin Alexander , Sebastiaan De Peuter.

## Exercise 8, due on Tuesday March 29 at 23:50.

### Contents
1. Problem 1: Minimize KL divergence using PyTorch
2. Problem 2: VB for a factor analysis model (1/2)
3. Problem 3: VB for a factor analysis model (2/2)

# Problem 1: Minimize KL divergence using PyTorch
PyTorch is a powerful auto-differentiation framework that allows us to do any optimization, as long as we can define the objective function and corresponding optimization variables. It has been widely used for Bayesian deep learning. In this exercise, we will study how to use PyTorch to fit a Gaussian distribution to a known Mixture of Gaussian by minimizing their KL divergence, and compare the difference between the forward and reverse form of the KL.

Recall that the KL divergence between two distributions $q(x)$ and $p(x)$ is defined as:

$$\text{KL}[q(x)|p(x)]=\int q(x)\log\frac{q(x)}{p(x)}dx.$$

This is typically called the **Reverse KL** which we have used before in the course (like in Variational Bayes). If the probability density functions of $q(x)$ and $p(x)$ are known, and we can get samples from $q(x)$, an unbiased estimator of KL divergence is:
$$\text{KL}[q(x)|p(x)]\approx\log\frac{q(x_i)}{p(x_i)}=\log q(x_i)-\log p(x_i),$$
where $x_i\sim q(x)$. We will use above estimator for this exercise.

There is also a **Forward KL**: $\text{KL}[p(x)|q(x)]$ defined as:

$$\text{KL}[p(x)|q(x)]=\int p(x)\log\frac{p(x)}{q(x)}dx,$$

which is used in other inference algorithms such as Expectration Propogation which is not within the scope of this course.

Let $p(x \mid \pi) = \pi \mathcal{N}(0,1)+(1-\pi)\mathcal{N}(8,1)$ where $\pi\sim\text{Bernoulli}(0.4)$ be the true mixture distribution which we want to fit using a Gaussian $q(x; \mu, \sigma)$. We want to estimate $\mu$ and $\sigma$ using both the forwared and reverse KL.

Complete the template below with the relevant code.  

In [ ]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributions as Dis
import matplotlib
import matplotlib.pyplot as plt

class Gaussian:
    """
    This represents q(x) 
    Gaussian distribution is parametrized by mean (mu) and standard deviation. The standard deviation is 
    parametrized as sigma = log(1 + exp(rho)) to make it positive all the time. A sample from the distribution
    can be obtained by first sampling from a unit Gaussian, shifting the samples by the mean and scaling by the 
    standard deviation: w = mu + log(1 + exp(rho)) * epsilon
    """
    def __init__(self, mu, rho):
        self.mean = mu
        self.rho = rho

    @property
    def std_dev(self):
        return torch.log1p(torch.exp(self.rho))

    def sample(self, num_samples = 1):
        # Sample num_samples data points from Gaussian distribution
        # Return a tensor contains all the samples 
        
        # Sample num_samples datapoints from N(0,1) 
        epsilon = Dis.Normal(0,1).sample([num_samples])
        
        # Scale and shift epsilon
        # samples = ? # EXERCISE
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        return samples
    
    def logprob(self, samples):
        # Compute the log probability of each sample under Gaussian distribution
        # Return a tensor containing the log probability of all samples        
        
        # logp = ? # EXERCISE
        # YOUR CODE HERE
        raise NotImplementedError()
        
        return logp
    
class MoG:
    """
    This represents p(x).
    In this example, mixture of two Gaussian distribution is constructed by 2 Gaussian distributions 
    N(0,2) and N(8,1), and each datapoint is from N(0,2) with probability p = 0.4 and from N(8,1) with 
    probability 0.6.
    """
    def __init__(self, mu_1=0., sigma_1=1., mu_2=8., sigma_2=1., prob = 0.4):
        self.mean_1 = torch.tensor(mu_1)
        self.sigma_1 = torch.tensor(sigma_1)
        self.mean_2 = torch.tensor(mu_2)
        self.sigma_2 = torch.tensor(sigma_2)
        self.prob = torch.tensor(prob)

    def sample(self, num_samples = 1):
        # Sample num_samples data points from MoG distribution
        # Return a tensor contains all the samples
        
        # sample from N(0, 2)
        # sample form N(8, 1)
        # sample from Bern(0.4)
        # Combine the three to from a sample form mixture
        # sample_gaussian_1 = ? # EXERCISE
        # sample_gaussian_2 = ? # EXERCISE
        # sample_bernoulli = ? # EXERCISE
        # samples  = ? # EXERCISE
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        return samples
    
    def logprob(self, samples):
        
        # Compute the log probability of each sample under the MoG distribution
        # Return a tensor containing the log probability of all samples
        
        # logp = ? # EXERCISE
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        return logp    

class KL_divergence(nn.Module):
    def __init__(self):
        super(KL_divergence, self).__init__()
        # define the mean and standard deviation as parameters, and initialization
        self.mu = nn.Parameter(torch.Tensor(1).uniform_(-2., 12.))
        self.rho = nn.Parameter(torch.Tensor(1).uniform_(1.0, 5.0))
        
        self.gaussian = Gaussian(self.mu, self.rho)
        self.mog = MoG()
    
    def compute_forwardKL(self):
        num_samples = torch.tensor(1000)
        
        # compute the forward KL divergence between p and q of num_samples data points
        # Return the estimated forward KL divergence
        
        
        # sample form MoG
        # compute forware KL 
        
        # samples = ? # EXERCISE
        # fkl = ? # EXERCISE
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        return fkl
    
    def compute_reverseKL(self):
        num_samples = torch.tensor(1000)
        # compute the reverse KL divergence between p and q with num_samples data points
        # Return the estimated reverse KL divergence
        
        # sample form Gaussian
        # compute reverse KL 
        
        # samples = ? # EXERCISE
        # rkl = ? # EXERCISE
        
        # YOUR CODE HERE
        raise NotImplementedError()
        return rkl

# Optimize the KL by using gradient descent
def optimization(kl, forward = False, learning_rate = 0.1, num_epoch = 1000):
    parameters = set(kl.parameters())
    optimizer = optim.Adam(parameters, lr = learning_rate, eps=1e-3)
    
    for epoch in range(num_epoch):
        optimizer.zero_grad()
        if forward:
            loss = kl.compute_forwardKL()
        else:
            loss = kl.compute_reverseKL()
            
        loss.backward()
        optimizer.step()
        
        if (epoch % 100) == 0:
            print('EPOACH %d: KL: %.4f.'% (epoch+1, loss))

print('Optimizing reverse KL')
torch.manual_seed(0)
kl_reverse = KL_divergence()
optimization(kl_reverse, forward = False)
Gaussian_reverse= kl_reverse.gaussian

print('Optimizing forward KL')
kl_forward = KL_divergence()
optimization(kl_forward, forward= True)
Gaussian_forward = kl_forward.gaussian

# Plot the pdf of Gaussian fitted from forward KL and reverse KL, and also the ground truth pdf from MoG
x_plot = torch.linspace(-5., 15., 1000)
density_mog = torch.exp(kl_forward.mog.logprob(x_plot)).detach().numpy()
density_Gaussian_forward = torch.exp(Gaussian_forward.logprob(x_plot)).detach().numpy()
density_Gaussian_reverse = torch.exp(Gaussian_reverse.logprob(x_plot)).detach().numpy()

fig, ax = plt.subplots()
ax.plot(x_plot, density_mog)
ax.plot(x_plot, density_Gaussian_forward)
ax.plot(x_plot, density_Gaussian_reverse)

ax.legend(('MoG Density','Forward KL', 'Reverse KL'))

# Problem 2: VB for a factor analysis model (1/2)

The data set consists of $D$-dimensional vectors $\mathbf{x}_{n}\in \mathbb{R}^{D},$ for $n=1,\ldots,N$. We model the data using factor analysis with $K$-dimensional factors $\mathbf{z}_{n}\in\mathbb{R}^{K}$. In detail, the
model is specified as follows:
\begin{align*}
\mathbf{x}_{n} &  \sim\mathcal{N}_{D}(\mathbf{Wz}_{n},\text{diag}
(\mathbf{\psi})^{-1}),\quad n=1,\ldots,N,\\
\psi_{d} &  \sim\text{Gamma}(a,b),\quad d=1,\ldots,D,\\
\mathbf{w}_{d} &  \sim\mathcal{N}_{K}(\mathbf{0,}\alpha\mathbf{I}),\quad
d=1,\ldots,D,\\
\mathbf{z}_{n} &  \sim\mathcal{N}_{K}(\mathbf{0,I}),\quad n=1,\ldots,N.
\end{align*}
Here, $\mathbf{W}$ is a $D\times K$ factor loading matrix and $\mathbf{w}_{d}$ is the $d$th row of $\mathbf{W}$ written as a column vector. Parameter $\psi_{d}^{-1}$ is the variance for the $d$th dimension in the observed data
and diag$(\psi)$ denotes a diagonal matrix with elements $\mathbf{\psi} =(\psi_{1},\ldots,\psi_{D})^{T}$ on the diagonal.

We approximate the posterior $p(\mathbf{\psi},\mathbf{Z},\mathbf{W} |\mathbf{X})$ using the mean-field approximation: 

$$ 
q(\Theta)=\prod_{d=1}^{D}q(\mathbf{w}_{d})\prod_{n=1}^{N}q(\mathbf{z}_{n})\prod_{d=1}^{D}q(\psi_{d}).
$$


__1__ Write the logarithm of the joint distribution, $\log p(\mathbf{\psi},\mathbf{Z},\mathbf{W},\mathbf{X})$.

__2__ Remove from the logarithm of the joint distribution all terms that do not depend on $\mathbf{z}_{n}$.

__3__ Show that the updated factor $q(\mathbf{z}_{n})$ is equal to

$$
q(\mathbf{z}_{n})=\mathcal{N}_{K}(\mathbf{\mu}_{n},\mathbf{K}_{n}),
$$

where

\begin{align*}
\mathbf{K}_{n} &  =\left[  I+\sum_{d=1}^{D}\left\langle \psi_{d}\right\rangle
\left\langle \mathbf{w}_{d}\mathbf{w}_{d}^{T}\right\rangle \right]  ^{-1}
\quad\text{and}\\
\mathbf{\mu}_{n} &  =\mathbf{K}_{n}\left\langle \mathbf{W}^{T}\right\rangle
\text{diag}(\left\langle \mathbf{\psi}\right\rangle )\mathbf{x}_{n}.
\end{align*}

Here $\left\langle \mathbf{\cdot}\right\rangle $ is used as a shorthand for the expectation of a variable with respect to its factor, e.g., $\left\langle \mathbf{\psi}\right\rangle =\mathbb{E}_{q(\mathbf{\psi})}[\mathbf{\psi}]$ etc.

__Hint 1:__ Try to write the log joint as 

$$
-\frac{1}{2}\mathbf{z}_{n}^{T}\mathbf{Az}_{n}+\mathbf{b}^{T}\mathbf{z}_{n}
$$

for some $\mathbf{A}$ and $\mathbf{b}$, after which you can apply the 'completing the square' technique.

__Hint 2:__ Suppose $\mathbf{A}$ is an $N\times M$ matrix. Further suppose that $\mathbf{D}$ is an $N\times N$ diagonal matrix, $\mathbf{D} =$diag$(d_{1},\ldots,d_{N})$. Then $\mathbf{A}^{T}\mathbf{DA}$ can be written
as

$$
\mathbf{A}^{T}\mathbf{DA=}\sum_{n=1}^{N}d_{n}\mathbf{a}_{n}\mathbf{a}_{n}^{T},
$$

where $\mathbf{a}_{n}$ is the $n$th row of $\mathbf{A}$ written as a column vector.

__Hint 3:__ Recall that expectation is a linear operator, i.e. $\mathbb{E}(aX+bY)=a\mathbb{E}(X)+b\mathbb{E}(Y)$. Further, if some random variables $A$ and $B$ are independent, then $\mathbb{E}_{q(A)q(B)} (AB)=\mathbb{E}_{q(A)}(A)\mathbb{E}_{q(B)}(B)$.


Write your answer to Problem 2 here.

# Problem 3: VB for a factor analysis model (2/2)
For the factor analysis model considered in Problem 2, derive the update for factor $q(\mathbf{w}_{d})$. The updated factor should be given in terms of the following expectations: $\left\langle \psi_{d}\right\rangle ,\left\langle \mathbf{z}_{n}\right\rangle ,\left\langle \mathbf{z}_{n}\mathbf{z}_{n}^{T}\right\rangle $, which have been calculated using the current values of the other factors for all $d,n$.

__Hint__: A multivariate Gaussian with a diagonal covariance can be expressed as a product of independent univariate Gaussians, which allows you to simplify the formulas.


Write your answer to Problem 3 here.